### Creations Graphs

CREATE TABLE stop_times_complete (
    trip_id BIGINT,
    arrival_time TEXT,
    departure_time TEXT,
    real_arrival_time TEXT,
    real_departure_time TEXT,
    speed_calculated FLOAT,
    stop_id TEXT,
    stop_sequence INT,
    pickup_type INT,
    drop_off_type INT,
    date_format DATE NOT NULL

);

In [6]:
import pandas as pd
import numpy as np
from datetime import datetime,date,timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import psycopg2
import pandas.io.sql as psql
from sqlalchemy import create_engine
import sshtunnel 
import pandas as pd
import psycopg2.extras
from datetime import datetime
from pytz import timezone

In [6]:
#insert results to database table assessment_methods
def insert_into_db(df_methods):
    try:
        sshtunnel.SSH_TIMEOUT = 10.0
        with sshtunnel.SSHTunnelForwarder(
                ('3.83.88.154', 22),
                ssh_username='ec2-user',
                #CHANGE PATH TO rds-example.cer
                ssh_pkey='/Users/lauraforerocamacho/Downloads/rds-example.cer',
                remote_bind_address=('stib-mivb.cjo3cnpt9pci.us-east-1.rds.amazonaws.com', 5432)
        ) as tunnel:
                engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{db}'.format(
                        host='127.0.0.1',
                        port=tunnel.local_bind_port,     
                        db="stibmivbdb",
                        user="postgres",
                        password="stib-mivb-db1" ))

                df_methods.to_sql('assessment_methods', engine, if_exists='append',index=False)
    except Exception as e:
            print(e)

In [ ]:
def get_scheduled_stoptimes():
    
    #SQL statement
    sql = """Select distinct t.route_id,
    t.service_id,
    t.trip_id,
    t.trip_headsign,
    t.direction_id,
    st.arrival_time,
    st.departure_time,
    st.stop_id,
    r.route_type,
    c.start_date,
    c.end_date,
    c.monday,
    c.tuesday,
    c.wednesday,
    c.thursday,
    c.friday,
    c.saturday,
    c.sunday
    from trips t
    inner join stop_times st
    on t.trip_id = st.trip_id 
    inner join calendar c
    on t.service_id = c.service_id
    inner join routes r
    on t.trip_headsign = SUBSTRING(r.route_long_name from POSITION('-' IN r.route_long_name)+2
                        for char_length(r.route_long_name)-POSITION('-' IN r.route_long_name)+1)
    and r.route_id = t.route_id
    where c.end_date >= '2021-09-01' and
    st.date_format='2022-09-03' 
    order by st.arrival_time, st.trip_id, st.stop_sequence"""
    
    #database connection

    try:
        sshtunnel.SSH_TIMEOUT = 10.0
        with sshtunnel.SSHTunnelForwarder(
                ('3.83.88.154', 22),
                ssh_username='ec2-user',
                #CHANGE PATH TO rds-example.cer
                ssh_pkey='/Users/lauraforerocamacho/Downloads/rds-example.cer',
                remote_bind_address=('stib-mivb.cjo3cnpt9pci.us-east-1.rds.amazonaws.com', 5432)
        ) as tunnel:
                conn = psycopg2.connect(
                        host='127.0.0.1',
                        port=tunnel.local_bind_port,     
                        database="stibmivbdb",
                        user="postgres",
                        password="stib-mivb-db1")

                cur = conn.cursor()
                #perform sql and write result to dataframe
                cur.execute(sql)
                field_names = [i[0] for i in cur.description]
                query_results = pd.DataFrame(cur.fetchall(),columns=field_names)
                #close connection
                cur.close()
                
                return query_results
    except Exception as e:
            print(e)

In [6]:
def get_data_from_vehiclepositions():
    
    #SQL statement
    sql = """Select time, lineid, directionid, pointid, min(distancefrompoint)
    from vehiclepositioncomplete
    group by lineid, directionid, pointid
    order by time, lineid, directionid, pointid"""
    
    #database connection

    try:
        sshtunnel.SSH_TIMEOUT = 10.0
        with sshtunnel.SSHTunnelForwarder(
                ('3.83.88.154', 22),
                ssh_username='ec2-user',
                #CHANGE PATH TO rds-example.cer
                ssh_pkey='/Users/lauraforerocamacho/Downloads/rds-example.cer',
                remote_bind_address=('stib-mivb.cjo3cnpt9pci.us-east-1.rds.amazonaws.com', 5432)
        ) as tunnel:
                conn = psycopg2.connect(
                        host='127.0.0.1',
                        port=tunnel.local_bind_port,     
                        database="stibmivbdb",
                        user="postgres",
                        password="stib-mivb-db1")

                cur = conn.cursor()
                #perform sql and write result to dataframe
                cur.execute(sql)
                field_names = [i[0] for i in cur.description]
                query_results = pd.DataFrame(cur.fetchall(),columns=field_names)
                #close connection
                cur.close()
                
                return query_results
    except Exception as e:
            print(e)

In [ ]:
df_methods=pd.read_csv("./realability/speed_stops.csv")

In [ ]:
df_vehiclepostions = get_data_from_vehiclepositions()

In [80]:
df_vehiclepostions["time"]= pd.to_datetime(df_vehiclepostions["time"], unit='ms')
df_vehiclepostions["time"]=[i.tz_localize("utc") for i in df_vehiclepostions["time"]]
df_vehiclepostions["time"]=[i.tz_convert("Europe/Brussels") for i in df_vehiclepostions["time"]]
